In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-dae8a89c-8416-6b29-d854-d37fdad6ac71)


In [2]:
!pip install -q kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d nunenuh/pytorch-challange-flower-dataset

100% 330M/330M [00:18<00:00, 20.5MB/s]
100% 330M/330M [00:18<00:00, 18.5MB/s]


In [ ]:
!unzip /content/pytorch-challange-flower-dataset.zip

In [5]:
import pandas as pd
import numpy as np


In [6]:
import os

In [7]:
train = pd.DataFrame(np.zeros(6552, ))
train['path'] = 'z'
z = 0
for i in os.listdir('/content/dataset/train'):
  train[str(i)] = 0
  for x in os.listdir('/content/dataset/train/'+str(i)):
    train['path'][z] = '/content/dataset/train/' + str(i)+ '/' + x
    train[str(i)][z] = 1
    z += 1

    

<ipython-input-7-a22db5d34004>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['path'][z] = '/content/dataset/train/' + str(i)+ '/' + x
<ipython-input-7-a22db5d34004>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[str(i)][z] = 1
<ipython-input-7-a22db5d34004>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  train[str(i)] = 0


In [8]:
val = pd.DataFrame(np.zeros(818, ))
val['path'] = 'z'
z = 0
for i in os.listdir('/content/dataset/valid'):
  val[str(i)] = 0
  for x in os.listdir('/content/dataset/valid/'+str(i)):
    val['path'][z] = '/content/dataset/valid/' + str(i) +'/'+ x
    val[str(i)][z] = 1
    z += 1

<ipython-input-8-6237684e6000>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['path'][z] = '/content/dataset/valid/' + str(i) +'/'+ x
<ipython-input-8-6237684e6000>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val[str(i)][z] = 1
<ipython-input-8-6237684e6000>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  val[str(i)] = 0


In [9]:
!pip install timm > /dev/null

In [10]:
import cv2
import matplotlib.pyplot as plt

import torch 
from torch import nn
from torchsummary import summary

import timm
from torch.optim import Adam, lr_scheduler

from myGen import MyGen

from sklearn.metrics import roc_auc_score


In [11]:
train.to_csv('/content/train.csv', index=False)
val.to_csv('/content/val.csv', index=False)

In [12]:
save_weights_path = '/content/Z_resnet50_320.pth'
load_weights_path = '/content/Z_resnet50_320.pth'

In [ ]:
timm.list_models(pretrained=True)

In [14]:
model = timm.create_model('resnet50', num_classes=102, pretrained=True)
model = model.cuda()

# summary(model, (3, 224, 224), batch_size=64, device='cuda')

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet50_a1_0-14fe96d1.pth" to /root/.cache/torch/hub/checkpoints/resnet50_a1_0-14fe96d1.pth


In [15]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [16]:
root = ''
f = os.listdir('/content/dataset/train')
trainGen = MyGen(image_root_path=root, 
                 csv_path='train.csv', 
                 image_size=256, 
                 col_to_train = f ,
                 frontal=False)

valGen =  MyGen(image_root_path=root, 
                 csv_path='val.csv', 
                 image_size=256, 
                 col_to_train = f,
                 frontal=False)

trainloader =  torch.utils.data.DataLoader(trainGen, batch_size=64, shuffle=True)
testloader =  torch.utils.data.DataLoader(valGen, batch_size=1, shuffle=True)

# define loss & optimizer
Loss = nn.BCELoss()

optimizer = Adam(model.parameters(), lr=1e-3)

scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)  # Decay LR by a factor of 0.5 every 5 epochs

#model.load_state_dict(torch.load(load_weights_path))

# training
const_threshold = 5

val_auc_mean = 0
best_val_auc = 0
const_cnt = 0

for epoch in range(5):

    for idx, data in enumerate(trainloader,1):
      train_data, train_labels = data
      train_data, train_labels  = train_data.cuda(), train_labels.cuda()
      y_pred = model(train_data)
      R = nn.Softmax(dim = 1)
      y_pred = R(y_pred)
      loss = Loss(y_pred, train_labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
        
      # validation  
      if idx % 30 == 0:
         model.eval()
         with torch.no_grad():    
              test_pred = []
              test_true = [] 
              for data in testloader:
                  test_data, test_labels = data
                  test_data = test_data.cuda()
                  y_pred = model(test_data)
                  R = nn.Softmax(dim = 1)
                  y_pred = R(y_pred)
                  test_pred.append(y_pred.cpu().detach().numpy())            
                  test_true.append(test_labels.numpy())
            
              test_true = np.concatenate(test_true)
              test_pred = np.concatenate(test_pred)
              val_auc_mean =  roc_auc_score(test_true, test_pred) 
              model.train()

              if best_val_auc < val_auc_mean:
                  const_cnt = 0
                  scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
                  best_val_auc = val_auc_mean
                  torch.save(model.state_dict(), save_weights_path)
              else:
                  const_cnt += 1
                  scheduler.step()            

              if (const_cnt > const_threshold):
                  const_cnt = 0 
                  scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
                  model.load_state_dict(torch.load(save_weights_path))
                               
              lr = get_lr(optimizer)

              print ('Epoch=%s, BatchID=%s, Val_AUC=%.4f, Best_Val_AUC=%.4f, lr=%f' %(epoch, idx, val_auc_mean, best_val_auc, lr))

Epoch=0, BatchID=30, Val_AUC=0.8593, Best_Val_AUC=0.8593, lr=0.001000
Epoch=0, BatchID=60, Val_AUC=0.9453, Best_Val_AUC=0.9453, lr=0.001000
Epoch=0, BatchID=90, Val_AUC=0.9728, Best_Val_AUC=0.9728, lr=0.001000
Epoch=1, BatchID=30, Val_AUC=0.9881, Best_Val_AUC=0.9881, lr=0.001000
Epoch=1, BatchID=60, Val_AUC=0.9936, Best_Val_AUC=0.9936, lr=0.001000
Epoch=1, BatchID=90, Val_AUC=0.9963, Best_Val_AUC=0.9963, lr=0.001000
Epoch=2, BatchID=30, Val_AUC=0.9884, Best_Val_AUC=0.9963, lr=0.001000
Epoch=2, BatchID=60, Val_AUC=0.9976, Best_Val_AUC=0.9976, lr=0.001000
Epoch=2, BatchID=90, Val_AUC=0.9926, Best_Val_AUC=0.9976, lr=0.001000
Epoch=3, BatchID=30, Val_AUC=0.9969, Best_Val_AUC=0.9976, lr=0.001000
Epoch=3, BatchID=60, Val_AUC=0.9982, Best_Val_AUC=0.9982, lr=0.001000
Epoch=3, BatchID=90, Val_AUC=0.9918, Best_Val_AUC=0.9982, lr=0.001000
Epoch=4, BatchID=30, Val_AUC=0.9984, Best_Val_AUC=0.9984, lr=0.001000
Epoch=4, BatchID=60, Val_AUC=0.9974, Best_Val_AUC=0.9984, lr=0.001000
Epoch=4, BatchID=90,

In [22]:
testGen = MyGen(image_root_path=root, 
                 csv_path='train.csv', 
                 image_size=256, 
                 col_to_train = f ,
                 frontal=False)

testloader =  torch.utils.data.DataLoader(testGen, batch_size=1, shuffle=False)

In [ ]:
model.load_state_dict(torch.load(save_weights_path))

In [23]:
model.eval()
with torch.no_grad():    
    test_pred = []
    test_true = [] 
    for data in testloader:
        test_data, test_labels = data
        test_data = test_data.cuda()
        y_pred = model(test_data)
        R = nn.Softmax(dim = 1)
        y_pred = R(y_pred)
        test_pred.append(y_pred.cpu().detach().numpy())
        test_true.append(test_labels.numpy())
  
    test_true = np.concatenate(test_true)
    test_pred = np.concatenate(test_pred)

In [24]:
labels = f

results = pd.DataFrame(index=labels)


scores = []
for i in range(102):
    score = roc_auc_score(test_true[:, i], test_pred[:, i])
    scores.append(score)
    
results['AUC'] = scores

In [25]:
results.head(102)

,AUC
98,1.000000
62,1.000000
7,1.000000
54,1.000000
21,1.000000
...,...
51,1.000000
28,1.000000
91,1.000000
60,1.000000


In [26]:
results['AUC'].mean()

0.9999995802002526